# <center>**基于流的模型（Flow-based Models）**</center>

### **1 流模型简介**

流模型（Flow-based Models）的核心思想是通过可逆函数构建一个模型。这类模型与生成对抗网络（GANs）的主要区别在于，流模型中没有判别器；而与变分自动编码器（VAEs）相比，流模型并不使用逆映射的近似值，而是通过真实的逆映射构造模型。这是基于**变量变换定理**实现的，允许从一个概率密度转变为另一个概率密度。

更具体地说，我们对一系列分布 $ z_i \sim p_i(z_i) $ 进行建模，其中每一个 $ z_{i+1} $ 是通过一个可逆变换 $ f_{i \rightarrow i+1} $ 从 $ z_i $ 得到的。简而言之，我们有如下关系（见图 1）：

$$
z_i \sim p_i(z_i), \quad z_{i+1} = f_{i \rightarrow i+1}(z_i), \quad z_i = f_{i+1 \rightarrow i}(z_{i+1})
$$

这里，$ f_{i \rightarrow i+1} $ 是解码器，而 $ f_{i+1 \rightarrow i} $ 是编码器，两者互为逆函数：

$$
f_{i+1 \rightarrow i} = f_{i \rightarrow i+1}^{-1} \quad \text{and} \quad f_{i+1 \rightarrow i}^{-1} = f_{i \rightarrow i+1}
$$

这使我们能够通过多重积分中的**变量变换公式**，将 $ z_{i+1} $ 的分布表达为 $ z_{i} $ 的分布：

$$
p_{i+1}(z_{i+1}) = p_{i}(z_{i}) \left| \operatorname{det} \frac{d z_{i}}{d z_{i+1}} \right| = p_{i}(z_{i}) \left| \operatorname{det} \frac{d f_{i+1 \rightarrow i}}{d z_{i+1}} \right|
$$

这里利用了可逆函数的一个重要性质：

$$
\operatorname{det}(J_{f^{-1}}) = \operatorname{det}\left( \frac{\partial f^{-1}}{\partial x} \right) = \operatorname{det}\left( \frac{\partial f}{\partial y} \right)^{-1} = \operatorname{det}(J_{f})^{-1}
$$

其中，$ J_f $ 表示函数 $ f $ 的雅可比矩阵。

这表明在计算雅可比矩阵的行列式时，需要选择最简单的形式。

通过递推公式并使用观察变量 $ x = z_K $，我们可以得到以下关系：

$$
\log p(x) = \log p(z_K) = \log p(z_{K-1}) + \log \left| \det \frac{df_{K \rightarrow K-1}}{dz_K} \right|
$$

进一步展开：

$$
\log p(x) = \log p_0(f_{1,0} \circ \ldots \circ f_{K,K-1}(z_K)) + \sum_{i=0}^{K-1} \log \left| \det \frac{df_{i+1 \rightarrow i}}{dz_{i+1}} \right|
$$

因此，可以直接最大化数据的似然函数，而不需要像 GAN 那样依赖判别器，也无需像 VAE 那样使用 ELBO（变分下界）的近似。这一点如图 2 所示。然而，为此付出的代价是函数的表达能力降低，因为对每个变换函数 $ f_i $ 有以下要求：

- 必须是可逆的（用于似然函数最大化时计算 $ z_0 $，以及在生成过程中计算 $ x = z_K $），
- 其雅可比矩阵的行列式必须是可计算的。

### **三角矩阵的行列式性质**

关于行列式，大多数流模型会设计雅可比矩阵为三角矩阵。在这种情况下，行列式的计算非常简单：

$$
\left| \begin{pmatrix}
d_1 & ? & \cdots & ? \\
0 & d_2 & ? & \vdots \\
\vdots & \ddots & \ddots & ? \\
0 & \cdots & 0 & d_n
\end{pmatrix} \right| = \prod_{i=1}^n d_i
$$

即三角矩阵的行列式等于对角线上元素的乘积。

### **矩阵乘积的行列式**

如果 $ A $ 和 $ B $ 是两个 $ n $ 阶方阵（$ A, B \in \mathbb{R}^{n \times n} $），那么有以下性质：

$$
\det(AB) = \det(A) \times \det(B)
$$

即矩阵乘积的行列式等于各矩阵行列式的乘积。

### **2. 代码模型**

您将使用文件 **`utils.py`**，其中包含以下内容：

- **`FlowModule` 类**：  
  一个通用类，定义了两个函数：  
  - $ f_{i \rightarrow i+1} $（解码器）和 $ f_{i+1 \rightarrow i} $（编码器）。  
  - 此类还包含一个方法 **`check(x: torch.Tensor)`**，用于验证逆映射是否正确计算：
    - 您可以在使用模块之前使用此方法进行验证。
    - 此外，您还可以借助该方法检查 $ \log|\det J_f| $ 是否正确计算。

- **`FlowSequential` 类**：  
  类似于 `torch.nn.Sequential`，支持顺序调用解码器和编码器。

- **`FlowModel` 类**：  
  用于完整定义流模型，并集成先验分布。

在整个实验中，您需要定义继承自 **`FlowModule`** 的类（`FlowModule` 本身继承自 `torch.nn.Module`），这些类应定义以下功能：

- **解码器**：$ f_{i \rightarrow i+1} $  
- **编码器**：$ f_{i+1 \rightarrow i} $

编码器函数 **`encoder(y)`**（同样适用于解码器函数 `decoder`）应返回两个张量（注意，您处理的是批量数据）：

1. $ f_{i+1 \rightarrow i}(y) $ 的值；  
2. 雅可比矩阵行列式的对数：$ \log\left|\det J_{f_{i+1 \rightarrow i}}\right| $。

**优化**通过梯度下降实现（如反向传播）。

### **3. 热身练习：仿射变换**

第一个变换非常简单：这是一个仿射变换，用于改变随机变量的方差和均值。

$$
f(y; s, t) = y \odot \exp(s) + t
$$

其中，$ s $ 和 $ t $ 是参数，$ \odot $ 表示逐元素相乘。

### **任务**

将 $ z_0 \sim \mathcal{N}(0, 1) $ 转换为 $ x = z_1 \sim \mathcal{N}(\mu, \sigma) $。  
比较学习到的参数值（均值和方差）。

### **提示**

- 使用 **`torch.distributions`** 定义先验分布。
- 特别是，可以使用 **`Independent`** 定义具有对角协方差的多元正态分布。

### **4. Glow 模型（第一部分）**

仿射变换功能有限；为了学习更复杂的概率分布，我们将实现 **Glow** 模型，该模型基于以下三种操作（以及之后将介绍的一些其他操作）：

#### **1. ActNorm**
一种仿射变换，与第 3 节中的类似，其参数在第一个批次时会被初始化，以确保输出分布是**中心化**和**标准化**的。

#### **2. Affine Coupling Layer**
对一半的数据进行仿射变换，另一半的数据用来参数化变换。设 $ x $ 为当前输入值，其维度为 $ 2 \times l $：

$$
\begin{aligned}
s &= \text{MLP}_{\text{scale}}(x_{1:l}) \\
t &= \text{MLP}_{\text{shift}}(x_{1:l}) \\
y_{1:l} &= x_{1:l} \\
y_{l+1:2l} &= x_{l+1:2l} \odot \exp(s) + t
\end{aligned}
$$

- 其中：
  - $ x_{1:l} $ 是前半部分的数据；
  - $ x_{l+1:2l} $ 是后半部分的数据；
  - $\text{MLP}_{\text{scale}}$ 和 $\text{MLP}_{\text{shift}}$ 是两个多层感知机（MLP），分别用于生成比例 $ s $ 和偏移 $ t $。

为了能够对 $ x $ 向量的两部分进行变换，每一层都会交替选择哪个部分的维度进行仿射变换：
- 第 $ l+1 : 2l $ 部分进行仿射变换；
- 第 $ 1 : l $ 部分保持不变。

这种交替方式确保了输入向量的每一部分都会被充分学习。

#### **3. Convolution 1x1 inversible**

一个 $1\times 1$ 卷积（即1核）对应于输入通道的线性变换。在这个练习中，我们有2个输入（和输出）通道，因此 $W$ 是一个 $2 \times 2$ 的矩阵。当我们处理图像时，将会有与“点”数量相同的变换（在这里，我们只有一个点）。在所有情况下，变换对应于（对于每个点）：
$$
y = Wx
$$
其中$W$是一个可逆矩阵。

为了在优化过程中保持这一特性，我们采取以下步骤：通过 $LU$ 分解来分解矩阵 $W$，即
$$
W = PLU
$$
其中$P$是一个置换矩阵，$L$ 是一个下三角矩阵，$U$ 是一个上三角矩阵（$L$ 的对角线仅由1组成以保证唯一性）。我们隔离对角线以得到
$$
W = P(L' + Id)(U' + S) \qquad (1)
$$
其中 $L'$ 和 $U'$ 是具有零对角线的三角矩阵，$S$ 是一个对角矩阵。

从方程式(1)出发，我们可以计算 $|\det(W)|$ ，记住在矩阵中反转两行（或列）会使行列式乘以-1。

在 `utils.py` 文件中，我们提供了用于初始化参数并根据这些参数计算矩阵 $ W $ 的代码。  
您可以使用 `torch.inverse` 函数来实现不同的功能。


### **数据集**

首先，使用来自 sklearn 的数据集（例如月亮数据集）测试一个 Glow 模型（10 层交替结构）。使用 10 层 **ActNorm**、**Affine Coupling** 和 **Convolution1x1** 的交替结构。您将使用在 `utils.py` 中定义的 MLP 作为计算变换的函数（用于 Affine Coupling）。

```python
from sklearn import datasets
# data, _ = datasets.make_moons(
n_samples=n_samples, shuffle=True, noise=0.05, random_state=0
)
```

您应该能够得到类似于图 3 中的解决方案：
- 图 3a：经典的高斯先验，
- 图 3b：两个高斯混合的先验。

### **5. Bonus ：Glow（第二部分）**

#### **扩展到 2D**

为了将我们刚刚学到的内容应用到图像（即张量 $ w \times h \times c $）上，直接应用之前看到的操作不再适用；我们需要利用卷积网络的 2D 对称性。为此，之前看到的三种操作需要进行适应，以便能够处理图像 $ x \in \mathbb{R}^{h \times w \times c} $：

- **ActNorm/卷积**：相同的变换将应用于张量 $ x_{ij} $ 的每个元素。
- **Affine Coupling**：对于每个元素 $ x_{ij} $，计算并应用变换（这不是全局操作）。

#### **多尺度（压缩/拆分）**

我们定义了另外两个操作。第一个 **（压缩）** 基于在通道维度上压缩空间：一个张量从 $ \mathbb{R}^{c \times w \times h} $ 变成 $ \mathbb{R}^{4c \times \frac{h}{2} \times \frac{w}{2}} $ 的张量。比如，对于一个 $ 1 $ 通道的 $ 2 \times 2 $ 像素图像，压缩后会得到 $ 1 $ 个像素，但有 $ 4 $ 个通道。这是一种变相的池化操作。

为了保持快速的计算速度并利用图像的不同尺度，假设对于 **拆分** 操作，表示的部分将不再被修改。需要注意的是，这个操作是应用于通道维度的，即将一个 $ \mathbb{R}^{c \times h \times w} $ 的张量转换成一个 $ \mathbb{R}^{\frac{c}{2} \times h \times w} $ 的张量。因此，拆分操作定义为：

$$
z_{i-1} = \text{concat}(\text{channels}(z_i, 1: c/2); f_{i \rightarrow i-1}(\text{channels}(z_i, c/2, c)))
$$

其中，$\text{channels}(z, \text{range})$ 返回张量 $ z $ 中在指定范围内的通道，$\text{concat}$ 会按照通道维度将输入的张量拼接在一起。

这些操作在图 5 中有示意图表示。对于 CIFAR-10 数据集，您将使用 $ L=3 $ 和 $ K=16 $。在用于 Affine Coupling 操作的网络中，您将使用一个由三层卷积（$ 3 \times 3 $, $ 1 \times 1 $, 然后是 $ 3 \times 3 $）组成的网络，使用 ReLU 激活函数，并且每层的通道数为 512（对于隐藏层）。

### **实验**

接下来，您将使用 Glow 模型在 CIFAR-10 数据集上进行实验：

```python
from torchvision import datasets, transforms

train_dataset = datasets.CIFAR10(
    root='./cifar_data/', train=True,
    transform=transforms.ToTensor(), download=True
)
```

### **一些有用的函数**

为了绘制一系列样本（1D 或 2D），您可以使用在 `utils.py` 文件中定义的 `scatterplots` 函数。

### **参考文献**

[1] Kingma, D. P., and Dhariwal, P. Glow: Generative Flow with Invertible 1x1 Convolutions. arXiv:1807.03039 [cs, stat] (2018年7月)。arXiv: 1807.03039.